In [5]:
import pandas as pd
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame

In [9]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2019, 1, 19)

df = web.DataReader("FB", "yahoo", start, end)
df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-01-14,146.570007,141.270004,142.000000,145.389999,20520300,145.389999
2019-01-15,150.679993,145.990005,146.009995,148.949997,24069000,148.949997
2019-01-16,149.649994,147.000000,149.000000,147.539993,18025700,147.539993
2019-01-17,149.000000,146.500000,146.949997,148.300003,15787900,148.300003
2019-01-18,152.429993,148.550003,149.750000,150.039993,31029600,150.039993
